In [7]:

import numpy as np
import matplotlib.pyplot as plt 
from matplotlib import animation, rc
from IPython.display import HTML

# set up figure, the axis, and the plot elements 
fig, ax = plt.subplots()
plt.close()

### Necessary parameters for the particle on a ring ###
### Radius in atomic units... electron mass is 1 atomic unit of mass
mu = 1 
R = 5.29 
I = mu * R**2
hbar = 1
L = 2*(np.pi)
# radius for benzene reference Heyrovska et al 2008

### create a numpy array 
## Appropriate range - 0 -> 2pi
x = np.linspace(0,2*(np.pi),500)

#parameters for plot -> change to appropriate range
ax.set_xlim((0, L))
ax.set_ylim((-5, 5))

line, = ax.plot([], [], lw=2)


def init():
    line.set_data([], [])
    return (line,)

def poly(x, a, b):
    fx = a * x**2 + b
    return fx

def Gauss_Packet(x, x0, sig, k0):
    ci = 0 + 1j
    T1 = 1 / (sig * np.sqrt(2 * np.pi))
    T2 = np.exp((-0.5) * ((x - x0)/sig)**2)
    T3 = np.exp(ci * k0 * x)
    return T1 * T2 * T3

### Particle-on-a-ring function!
def POR(x, n):
    ### normalize!
    return np.sqrt(1/(2*np.pi))*np.exp(1j*n*x)

def integrate(x, f_of_x):
    ### get the width of the rectagles 
    dx = x[1] - x[0]
    N = len(x) - 1
    integral = 0
    for i in range (1,N):
        h = f_of_x[i]
        A = h * dx
        integral = integral + A
        
    return integral 

### Particle on a ring energy eigenvalues!
def POR_En(n):
    return (hbar*n)
    
def POR_Ene(I, n):
    return (hbar**2 *n**2)/(2*I)

### return time-dependent part of energy eigenfunction in atomic units 
def POR_Time(I, n, t):
    ci = 0+1j
    En = POR_Ene(I, n)
    return np.exp(-ci*En*t)

### x0 and sig to appropriate values for the particle-on-a-ring system 
x0 = np.pi 
k0 = 0
sig = 0.1

gp = Gauss_Packet(x, x0, sig, k0)
gpstar = np.conj(gp)
P_of_x = gpstar * gp

### position eigenfunction generator!
def Position_Eigenfunction(x, x0, sig):
    ### need sqrt of -1!
    ci = 0+1j
    ### T1 will be the prefactor that is 1/(sigma * sqrt(2 * pi))
    T1 = 1/(sig * np.sqrt(2 * np.pi))
    ### T2 will be the Gaussian function, exp(-0.5 * ((x-x0)/sigma)^2)
    T2 = np.exp(-0.5 * ((x-x0)/sig)**2)
    ### T3 will be the complex exponential (aka the plane wave!)
    ### return the product of T1 * T2 * T3
    return T1 * T2 


### make an array of quantum numbers
m_array = np.linspace(-50,50,101)

### create an empty array of coefficients with 500 enteries 
cn = np.zeros(len(m_array), dtype=complex)
numerator = 0 
denominator = 0
gpe = np.zeros_like(gp)
for i in range(0,len(m_array)): 
    ### get the current quantum number
    n = m_array[i]
   ### 1. the function you want to integrate against comes from POR(x, n)
    ### 2. the range of n should include negative integers as well
    cn[i] = integrate(x, np.conj(POR(x, n))*gp)
    gpe = gpe + cn[i] * POR(x, n)
    


##plt.plot(x, gp, 'red')
#plt.plot(x, gpe, 'b--')
#plt.show()

pf = Position_Eigenfunction(x, np.pi, 0.1)
gp = pf
y_exp = np.zeros_like(gp)
for i in range(0,100):
    ### get energy eigenstate n
    psi = POR(x, m_array[i])
    ### multiply psi_n by gaussian wavepacket
    integrand = np.conj(psi)*gp
    ### get coefficient c_n from the integral of psi_n * Psi_gp
    cn[i] = integrate(x, integrand)
    y_exp = y_exp + cn[i] * psi


N_time = 200
n0 = np.zeros(1)
# animate function. this is called sequentially 
def animate(i):
    y = np.zeros(len(x),dtype=complex)
    if i<50:
        for j in range(0,101):
            ft = POR_Time(I, m_array[j], i/10)
            fx = POR(x, m_array[j])
            y= y + cn[j] * fx * ft 

    ### make position measurement!
    elif i==50:
        for j in range(0,100):
            ft = POR_Time(I, m_array[j], i/10.)
            fx = POR(x, m_array[j])
            y = y + cn[j] * fx * ft
        ### get probability density
        P = np.real(np.conj(y) * y)
        ### make sure P is noramlized
        norm = np.sum(P)
        P_norm = P / norm
        ### measure position and get position eigenvalue/eigenfunction
        p0 = np.random.choice(x, 1, p=P_norm)
        print(" Position measured to be at ",p0)
        pf = Position_Eigenfunction(x, p0[0], 0.1)
        y = pf
            
        ### now get NEW expansion coefficients given that the new state is pf!
        for j in range(0,100):
            ### get energy eigenstate n
            psi = POR(x, m_array[j])
            ### multiply psi_n by gaussian wavepacket
            integrand = np.conj(psi)*pf
            ### get coefficient c_n from the integral of psi_n * Psi_gp
            cn[j] = integrate(x, integrand)
            ft = POR_Time(I, m_array[j], (i-50)/10.)
            y = y + cn[j] * psi
    elif i<150:
        for j in range(0,100):
            ft = POR_Time(I, m_array[j], (i-50)/10.)
            fx = POR(x, m_array[j])
            y = y + cn[j] * fx * ft
            
    ### measure energy!
    elif i==150:
        pn = np.real(np.conj(cn) * cn)
        norm = np.sum(pn)
        pn_norm = pn/norm
        ### get random quantum number 
        nval = np.random.choice(m_array, 1, p=pn_norm)
        n0[0] = nval[0]
        En0 = POR_En(n0[0])
        print(" Randomly measured state", n0[0], "which has angular momentum ",En0)
        ft = POR_Time(I, n0[0], (i-150)/10)
        fx = POR(x, n0[0])
        y = fx * ft
    else:
        ft = POR_Time(I, n0[0], (i-150)/10)
        fx = POR(x, n0[0])
        y = fx * ft
        
    line.set_data(x, np.real(y))
    return (line,)

rc('animation', html='jshtml')

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=N_time, interval=100, blit=True)

# Note: below is the part which makes it work on colab 

anim

 Position measured to be at  [4.69664954]
 Randomly measured state -7.0 which has angular momentum  -7.0
